# SafeGraph SIGSPATIAL 2021 Workshop

## Understand Consumer Behavior With Verified Foot Traffic Data

**Background** Visitor and demographic aggregation data provide essential context on population behavior. How often a place of interest is visited, how long do visitors stay, where did they come from, and where are they going? The answers are invaluable in numerous industries. Building financial indicators, city and urban planning, public health indicators, or identifying your primary business competitors all require accurate, high quality population and POI data. 

**Objective** Our workshop’s objective is to provide professionals, researchers, and practitioners interested in deriving human movement patterns from location data. We use a sample of our Weekly and Monthly Patterns and Core Places products to perform market research on a potential new coffee shop location. We’ll address these concerns and more in building a market analysis proposal in real time. 

**Questions to Answer** 
- How far are customers willing to travel for coffee? 
- What location will receive the most visibility? 
- Where do most of the coffee customers come from?

## Notebook Setup

In [ ]:
spark = SparkSession. \
    builder. \
    appName('sigspatial2021'). \
    config("spark.serializer", "org.apache.spark.serializer.KryoSerializer"). \
    config("spark.kryo.registrator", "org.apache.sedona.core.serde.SedonaKryoSerializer"). \
    getOrCreate()

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.types import MapType, StringType, IntegerType

import pandas as pd

### Load in SafeGraph sample data from s3
The covers all coffee shops (`category_tag.contains("Coffee Shop")`) in Seattle (`county_FIPS.isin(["53033","53053","53061"]`), with multiple rows per POI corresponding to monthly foot traffic since the beginning of 2018 (1 month per row per POI).

Columns are SafeGraph Core, Geo, and Patterns pre-joined together with `placekey` as the join key.

In [ ]:
sample_csv_path = '../data/seattle_coffee_monthly_patterns/'
sample = (
  spark.read.option("header", "true").option("escape", "\"").csv(sample_csv_path)
  .withColumn('date_range_start', f.to_date(f.col('date_range_start')))
  .withColumn('date_range_end', f.to_date(f.col('date_range_end')))
  .withColumn('visitor_home_cbgs', f.from_json('visitor_home_cbgs', schema = MapType(StringType(), IntegerType())))
)

In [ ]:
sample.limit(10).toPandas().head()

## Exploratory Data Analysis and Visualization

### Visualize the coffee shops

In [ ]:
from pyspark.sql.window import Window
import geopandas as gpd
import folium

In [ ]:
w = Window().partitionBy('placekey').orderBy(f.col('date_range_start').desc())

cafes_latest = (
  sample
  # as our data improves, addresses or geocodes for a given location may change over time
  # use a window function to keep only the most recent appearance of the given cafe
  .withColumn('row_num', f.row_number().over(w))
  .filter(f.col('row_num') == 1)
  # select the columns we need for mapping
  .select('placekey', 'location_name', 'brands', 'street_address', 'city', 'region', 'postal_code', 'latitude', 'longitude', 'open_hours')
)

In [ ]:
# create a geopandas geodataframe
cafes_gdf = cafes_latest.toPandas()
cafes_gdf = gpd.GeoDataFrame(cafes_gdf, geometry = gpd.points_from_xy(cafes_gdf['longitude'], cafes_gdf['latitude']), crs = 'EPSG:4326')

In [ ]:
def map_cafes(gdf):
  
  # map bounds
  sw = [gdf.unary_union.bounds[1], gdf.unary_union.bounds[0]]
  ne = [gdf.unary_union.bounds[3], gdf.unary_union.bounds[2]]
  folium_bounds = [sw, ne]
  
  # map
  x = gdf.centroid.x[0]
  y = gdf.centroid.y[0]
  
  map_ = folium.Map(
    location = [y, x],
    tiles = "OpenStreetMap"
  )
  
  for i, point in gdf.iterrows():
    
    tooltip = f"placekey: {point['placekey']}<br>location_name: {point['location_name']}<br>brands: {point['brands']}<br>street_address: {point['street_address']}<br>city: {point['city']}<br>region: {point['region']}<br>postal_code: {point['postal_code']}<br>open_hours: {point['open_hours']}"
    
    folium.Circle(
      [point['geometry'].y, point['geometry'].x],
      radius = 40,
      fill_color = 'blue',
      color = 'blue',
      fill_opacity = 1,
      tooltip = tooltip
    ).add_to(map_)

  map_.fit_bounds(folium_bounds) 
  
  return map_

In [ ]:
map_ = map_cafes(cafes_gdf)
map_

## Analysis

### How far are people willing to travel for coffee?

In [ ]:
# the `distance_from_home` column tells us the median distance (as the crow flies), in meters, between the coffee shop and the visitors' homes 
# which coffee shop's visitors had the highest average median distance traveled since Jan 2018?

# outlier values in this column distort the histogram
# these outliers are likely due to a combination of (1) coffee shops in downtown areas that receive high numbers of out-of-town visitors and (2) quirks in the underlying GPS data
furthest_traveled = (
  sample
  .groupBy('placekey', 'location_name')
  .agg(f.mean('distance_from_home').alias('avg_median_dist_from_home'))
  .orderBy('avg_median_dist_from_home', ascending = False)
)

display(furthest_traveled)

In [ ]:
# most coffee shops' visitors' homes are <10km away
display(furthest_traveled.filter(f.col('avg_median_dist_from_home') < 25000))

In [ ]:
# but `distance_from_home` takes into account ALL visitors to the coffee shop, which as we saw above can distort values. When selecting a site for a coffee shop, we likely care more about how far visitors traveled from within Seattle
# we can compute this using Sedona
import sedona
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

SedonaRegistrator.registerAll(spark)

In [ ]:
# load the census block groups for Washington State
# filter it down to our three counties of interest in Seattle 
WA_cbgs = (
  spark.read.option('header', 'true').option('escape', "\"").csv('../data/wa_cbg.csv')
  .filter(f.col('GEOID').rlike('^(53033|53053|53061)'))
)
WA_cbgs.head()

In [ ]:
# transform the geometry column into a Geometry-type
WA_cbgs = (
  WA_cbgs
  .withColumn('cbg_geometry', f.expr("ST_GeomFromWkt(geometry)"))
  # we'll just use the CBG centroid
  .withColumn('cbg_geometry', f.expr("ST_Centroid(cbg_geometry)"))
  # since we'll be doing a distance calculation, let's also use a projected CRS - epsg:3857
  .withColumn('cbg_geometry', f.expr("ST_Transform(ST_FlipCoordinates(cbg_geometry), 'epsg:4326','epsg:3857', false)")) # ST_FlipCoordinates() necessary due to this bug: https://issues.apache.org/jira/browse/SEDONA-39
  .withColumnRenamed('GEOID', 'cbg')
  .withColumnRenamed('geometry', 'cbg_polygon_geometry')
)

In [ ]:
WA_cbgs.limit(10).toPandas().head()

In [ ]:
# Next let's prep our sample data
sample_seattle_visitors = (
  sample
  .select('placekey', f.explode('visitor_home_cbgs'))
  .withColumnRenamed('key', 'cbg')
  .withColumnRenamed('value', 'visitors')
#   # filter out CBGs with low visitor counts
  .filter(f.col('visitors') > 4)
  # filter down to only the visitors from Seattle CBGs
  .filter(f.col('cbg').rlike('^(53033|53053|53061)'))
  # aggregate up all the visitors over time from each CBG to each Cafe
  .groupBy('placekey', 'cbg')
  .agg(
    f.sum('visitors').alias('visitors')
  )
  # join back with most up-to-date POI information
  .join(
    cafes_latest.select('placekey', 'latitude', 'longitude'),
    'placekey'
  )
  # transform geometry column
  .withColumn('cafe_geometry', f.expr("ST_Point(CAST(longitude AS Decimal(24, 20)), CAST(latitude AS Decimal(24, 20)))"))
  .withColumn('cafe_geometry', f.expr("ST_Transform(ST_FlipCoordinates(cafe_geometry), 'epsg:4326','epsg:3857', false)"))
  # join with CBG geometries
  .join(WA_cbgs, 'cbg')
)

In [ ]:
sample_seattle_visitors.limit(10).toPandas().head()

In [ ]:
distance_traveled_SEA = (
  sample_seattle_visitors
  # calculate the distance from home in meters
  .withColumn('distance_from_home', f.expr("ST_Distance(cafe_geometry, cbg_geometry)"))
)
distance_traveled_SEA.createOrReplaceTempView('distance_traveled_SEA')

In [ ]:
q = '''
SELECT *
FROM (
  SELECT DISTINCT 
    placekey, 
    cbg,
    visitors,
    distance_from_home,
    posexplode(split(repeat(",", visitors), ","))
    FROM distance_traveled_SEA
)
WHERE pos > 0
'''

weighted_median_tmp = spark.sql(q)

In [ ]:
grp_window = Window.partitionBy('placekey')
median_percentile = f.expr('percentile_approx(distance_from_home, 0.5)')

In [ ]:
weighted_median_tmp.limit(10).toPandas().head()

In [ ]:
median_dist_traveled_SEA = (
  weighted_median_tmp
  .groupBy('placekey')
  .agg(median_percentile.alias('median_dist_traveled_SEA'))
)

In [ ]:
median_dist_traveled_SEA.filter(f.col('median_dist_traveled_SEA') < 25000).limit(10).toPandas().head()


In [ ]:
total_visits = (
  sample
  .groupBy('placekey')
  .agg(
    f.sum('raw_visit_counts').alias('total_visits'),
    f.sum('raw_visitor_counts').alias('total_visitors')
  )
)

In [ ]:
distance_traveled_final = (
  furthest_traveled
  .join(median_dist_traveled_SEA, 'placekey')
  .join(cafes_latest, ['placekey', 'location_name'])
  .withColumn('distance_traveled_diff', f.col('avg_median_dist_from_home') - f.col('median_dist_traveled_SEA'))
  # keep only the cafes with a meaningful sample - at least 1000 visits since 2018
  .join(total_visits, 'placekey')
  .filter(f.col('total_visits') > 1000)
  .orderBy('distance_traveled_diff', ascending = False)
)

In [ ]:
distance_traveled_final.limit(10).toPandas().head()

In [ ]:
distance_traveled_final.select('placekey').distinct().count()

In [ ]:
most_tourists = distance_traveled_final.limit(500).withColumn('visitor_type', f.lit('tourist'))
most_locals = distance_traveled_final.orderBy('distance_traveled_diff').limit(500).withColumn('visitor_type', f.lit('local'))

visitor_type = most_tourists.unionByName(most_locals)

# create a geopandas geodataframe
visitor_type_gdf = visitor_type.toPandas()
visitor_type_gdf = gpd.GeoDataFrame(visitor_type_gdf, geometry = gpd.points_from_xy(visitor_type_gdf['longitude'], visitor_type_gdf['latitude']), crs = 'EPSG:4326')

In [ ]:
def map_cafe_visitor_type(gdf):
  
  # map bounds
  sw = [gdf.unary_union.bounds[1], gdf.unary_union.bounds[0]]
  ne = [gdf.unary_union.bounds[3], gdf.unary_union.bounds[2]]
  folium_bounds = [sw, ne]
  
  # map
  x = gdf.centroid.x[0]
  y = gdf.centroid.y[0]
  
  map_ = folium.Map(
    location = [y, x],
    tiles = "OpenStreetMap"
  )
  
  for i, point in gdf.iterrows():
    
    tooltip = f"placekey: {point['placekey']}<br>location_name: {point['location_name']}<br>brands: {point['brands']}<br>street_address: {point['street_address']}<br>city: {point['city']}<br>region: {point['region']}<br>postal_code: {point['postal_code']}<br>visitor_type: {point['visitor_type']}<br>avg_median_dist_from_home: {point['avg_median_dist_from_home']}"
    
    folium.Circle(
      [point['geometry'].y, point['geometry'].x],
      radius = 40,
      fill_color = 'blue' if point['visitor_type'] == 'tourist' else 'red',
      color = 'blue' if point['visitor_type'] == 'tourist' else 'red',
      fill_opacity = 1,
      tooltip = tooltip
    ).add_to(map_)

  map_.fit_bounds(folium_bounds) 
  
  return map_

In [ ]:
# blue = touristy, red = locals
map_cafe_visitor_type(visitor_type_gdf)

### What location will receive the most visibility?

In other words, in what neighborhood do coffee shops get the most visits generally?

In [ ]:
WA_neighbs = (
  spark.read.option('header', 'true').option('escape', "\"").csv('../data/seattle_neighborhoods.csv')
  # transform the geometry column into a Geometry-type
  .withColumn('geometry', f.expr("ST_GeomFromWkt(geometry)"))
)
WA_neighbs.createOrReplaceTempView('WA_neighbs')

WA_neighbs.limit(10).toPandas().head()

In [ ]:
cafes_geo = cafes_latest.withColumn('cafe_geometry', f.expr("ST_Point(CAST(longitude AS Decimal(24,20)), CAST(latitude AS Decimal(24,20)))")).select('placekey', 'cafe_geometry')
cafes_geo.createOrReplaceTempView('cafes_geo')

In [ ]:
# perform a spatial join
q = '''
SELECT cafes_geo.placekey, WA_neighbs.S_HOOD as neighborhood, WA_neighbs.geometry
FROM WA_neighbs, cafes_geo
WHERE ST_Intersects(WA_neighbs.geometry, cafes_geo.cafe_geometry)
'''

cafe_neighb_join = spark.sql(q)

In [ ]:
cafe_neighb_join.limit(10).toPandas().head()

In [ ]:
# add the visit and visitor counts and aggregate up to the neighborhood
neighborhood_agg = (
  cafe_neighb_join
  .join(total_visits, 'placekey')
  .groupBy('neighborhood', f.col('geometry').cast('string').alias('geometry'))
  .agg(
    f.sum('total_visits').alias('total_visits'),
    f.sum('total_visitors').alias('total_visitors')
  )
)

In [ ]:
neighbs_gdf = (
  neighborhood_agg
  .toPandas()
)
neighbs_gdf = gpd.GeoDataFrame(neighbs_gdf, geometry = gpd.GeoSeries.from_wkt(neighbs_gdf['geometry']), crs = 'EPSG:4326')

In [ ]:
def map_neighbs(gdf):
  
  # map bounds
  sw = [gdf.unary_union.bounds[1], gdf.unary_union.bounds[0]]
  ne = [gdf.unary_union.bounds[3], gdf.unary_union.bounds[2]]
  folium_bounds = [sw, ne]
  
  # map
  x = gdf.centroid.x[0]
  y = gdf.centroid.y[0]
  
  map_ = folium.Map(
    location = [y, x],
    tiles = "OpenStreetMap"
  )
    
  gdf['percentile'] = pd.qcut(gdf['total_visits'], 100, labels=False) / 100
  
  folium.GeoJson(
      gdf[['neighborhood', 'total_visits', 'total_visitors', 'percentile', 'geometry']],
      style_function = lambda x: {
        'weight':0,
        'color':'blue',
        'fillOpacity': x['properties']['percentile']
      },
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['neighborhood', 'total_visits', 'total_visitors', 'percentile']
      )
    ).add_to(map_)

  map_.fit_bounds(folium_bounds) 
  
  return map_

In [ ]:
map_neighbs(neighbs_gdf)

### What home location has the most coffee-shop-goers?

In [ ]:
home_loc_most_cafe_visitors = (
  sample
  .select(f.explode('visitor_home_cbgs'))
  .withColumnRenamed('key', 'cbg')
  .withColumnRenamed('value', 'visitors')
  .groupBy('cbg')
  .agg(f.sum('visitors').alias('visitors'))
  .orderBy('visitors', ascending = False)
)

In [ ]:
# 40k visitors to Seattle coffee shops since Jan 2018 originated from CBG `530330082001`, just under twice as many as the next CBG.
home_loc_most_cafe_visitors.limit(10).toPandas().head()

In [ ]:
from shapely import wkt

In [ ]:
# Map of the top 1000 CBGs in terms of visitors' origins
home_loc_gdf = (
  home_loc_most_cafe_visitors
  .limit(1000)
  .join(
    WA_cbgs.select('cbg', 'cbg_polygon_geometry'),
    'cbg'
  )
  .toPandas()
)
home_loc_gdf = gpd.GeoDataFrame(home_loc_gdf, geometry = gpd.GeoSeries.from_wkt(home_loc_gdf['cbg_polygon_geometry']), crs = 'EPSG:4326')

In [ ]:
def map_cbgs(gdf):
  
  # map bounds
  sw = [gdf.unary_union.bounds[1], gdf.unary_union.bounds[0]]
  ne = [gdf.unary_union.bounds[3], gdf.unary_union.bounds[2]]
  folium_bounds = [sw, ne]
  
  # map
  x = gdf.centroid.x[0]
  y = gdf.centroid.y[0]
  
  map_ = folium.Map(
    location = [y, x],
    tiles = "OpenStreetMap"
  )
  
  gdf['quantile'] = pd.qcut(gdf['visitors'], 100, labels=False) / 100
  
  folium.GeoJson(
      gdf[['cbg', 'visitors', 'geometry', 'quantile']],
      style_function = lambda x: {
        'weight':0,
        'color':'blue',
        'fillOpacity': x['properties']['quantile']
      },
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['cbg', 'visitors', 'quantile']
      )
    ).add_to(map_)

  map_.fit_bounds(folium_bounds) 
  
  return map_

In [ ]:
# top 1000 home_cbgs by number of people who visited coffee shops in Seattle
map_cbgs(home_loc_gdf)

## Conclusion

Apache Sedona provides a familiar and straight-forward spatial SQL API for performing distributed spatial queries.